In [0]:
# ======================================================================================
# OBJETIVO: Enriquecer o conjunto de dados de casos com dados climáticos (temperatura e
#           precipitação) obtidos de uma API externa (Meteostat) de acordo com a
#           localização e data de cada notificação.
# ======================================================================================

# --- 0. INSTALAÇÃO DE BIBLIOTECAS (se necessário) ---
# Estas linhas instalam as bibliotecas necessárias para a execução do script.
# meteostat: para consultar a API de dados climáticos.
# tqdm: para exibir barras de progresso em processos longos.
%pip install meteostat tqdm
%restart_python

In [0]:
# --- 1. CONFIGURAÇÃO E INICIALIZAÇÃO ---

# Importação das bibliotecas e funções necessárias.
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, DoubleType, DateType, StringType
import pandas as pd
from datetime import datetime, date 
from meteostat import Point, Daily, Stations
from tqdm import tqdm

In [0]:
# Inicializa a sessão do Spark.
spark = SparkSession.builder.appName("EnriquecimentoClima").getOrCreate()

# Carrega a tabela previamente processada que contém os dados dos casos,
# incluindo as coordenadas geográficas (latitude e longitude) de cada município.
df_com_geo = spark.table("workspace.default.casos_unificados_br")

In [0]:
# Se configura la clave de la API para la librería meteostat.
# Es necesario para realizar consultas a su servicio.
METEOSTAT_API_KEY = "8a4b224e05msh7db26cbb8f4532bp1b931djsn4a9215830d1a"
Daily.key = METEOSTAT_API_KEY
Stations.key = METEOSTAT_API_KEY

In [0]:
# --- 2. OBTENÇÃO DE ESTAÇÕES CLIMÁTICAS PRÓXIMAS ---

# Para otimizar as chamadas à API, primeiro identificam-se as combinações únicas
# de localização (lat, lon) e data de notificação em todo o conjunto de dados.
# .toPandas() converte o resultado do Spark para um DataFrame do Pandas,
# já que a biblioteca meteostat opera sobre estruturas de dados do Pandas.
loc_datas_unicas_pd = df_com_geo.select("lat_municipio", "lon_municipio", "data_notificacao").distinct().toPandas().dropna()

# Das combinações únicas, extraem-se apenas as coordenadas geográficas únicas.
# O objetivo é encontrar uma estação climática por cada localização, não por cada data.
loc_unicas = loc_datas_unicas_pd[['lat_municipio', 'lon_municipio']].drop_duplicates().to_records(index=False)

# Dicionário para mapear cada coordenada (lat, lon) à sua estação mais próxima.
mapa_estacoes = {}
# Lista para guardar as localizações para as quais não foi encontrada uma estação.
loc_sem_estacao = []

# Itera sobre cada localização geográfica única para encontrar sua estação mais próxima.
# 'tqdm' exibe uma barra de progresso, útil para monitorar processos longos.
for lat, lon in tqdm(loc_unicas, desc="Buscando estações próximas"):
    try:
        stations = Stations().nearby(lat, lon).fetch(1)
        if not stations.empty:
            mapa_estacoes[(lat, lon)] = stations.index[0]
        else:
            loc_sem_estacao.append((lat, lon))
    except Exception:
        loc_sem_estacao.append((lat, lon))

In [0]:
# --- 3. AGRUPAMENTO DE CONSULTAS E DOWNLOAD DE DADOS CLIMÁTICOS ---

# Converte o mapa de estações em um DataFrame para facilitar a junção.
mapa_pd = pd.DataFrame(mapa_estacoes.items(), columns=['coords', 'station_id'])
mapa_pd[['lat_municipio', 'lon_municipio']] = pd.DataFrame(mapa_pd['coords'].tolist(), index=mapa_pd.index)

# Junta as datas de notificação com as estações encontradas.
df_com_estacoes = pd.merge(loc_datas_unicas_pd, mapa_pd, on=['lat_municipio', 'lon_municipio'], how='left').dropna(subset=['station_id'])

# Agrupa as datas por estação para fazer uma única chamada de API por estação.
consultas_por_estacao = df_com_estacoes.groupby('station_id')['data_notificacao'].agg(list)

# Lista para armazenar os DataFrames de dados climáticos obtidos.
lista_clima_pd = []
hoje = date.today()

for station_id, datas in tqdm(consultas_por_estacao.items(), desc="Baixando dados climáticos"):
    try:
        data_inicio = min(datas)
        data_fim = max(datas)
        
        # Lógica simplificada: Se a data final for no futuro, ajuste-a para hoje.
        if data_fim > hoje:
            data_fim = hoje
        
        # Apenas prossiga se o intervalo de datas for válido.
        if data_inicio <= data_fim:
            data = Daily(station_id, data_inicio, data_fim).fetch()
            
            if not data.empty:
                data['station_id'] = station_id
                lista_clima_pd.append(data)

    except Exception as e:
        print(f"Aviso: Não foi possível obter dados para a estação {station_id} no intervalo de {data_inicio} a {data_fim}. Erro: {e}")

In [0]:
# --- 4. JUNÇÃO DOS DADOS CLIMÁTICOS E GRAVAÇÃO DA TABELA FINAL ---

if not lista_clima_pd:
    print("Erro: Não foi possível obter nenhum dado climático da API. O processo será interrompido.")
else:
    # Concatena todos os resultados climáticos em um único DataFrame do Pandas.
    clima_pd_completo = pd.concat(lista_clima_pd).reset_index()[['time', 'station_id', 'tavg', 'prcp']]
    clima_pd_completo.rename(columns={'time': 'data_notificacao', 'tavg': 'temperatura_media', 'prcp': 'precipitacao'}, inplace=True)
    
    # Converte o mapa de estações para um DataFrame do Spark para a junção.
    mapa_estacoes_spark = spark.createDataFrame(mapa_pd)
    # Adiciona o ID da estação ao DataFrame principal de casos.
    df_com_station_id = df_com_geo.join(mapa_estacoes_spark, on=['lat_municipio', 'lon_municipio'], how='left')
    
    # Converte os dados climáticos para um DataFrame do Spark.
    clima_spark_df = spark.createDataFrame(clima_pd_completo)
    
    # Junta os dados climáticos ao DataFrame principal usando o ID da estação e a data da notificação.
    df_final_com_clima = df_com_station_id.join(clima_spark_df, on=['station_id', 'data_notificacao'], how='left')
    
    print("Salvando a tabela enriquecida em 'workspace.default.casos_completos_com_clima'...")
    df_final_com_clima.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("workspace.default.casos_completos_com_clima")    
    
    count_final = df_final_com_clima.count()
    print(f"--- Processo concluído. Foram salvos {count_final} registros em 'casos_completos_com_clima'. ---")